In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv3D
from keras.layers import Conv3DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
import os
!pip install pypianoroll
import pypianoroll


     |████████████████████████████████| 5.6MB 26.4MB/s 
     |████████████████████████████████| 61kB 9.9MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-cp37-none-any.whl size=5591954 sha256=a2e4d420957391dbeee054f4882adab968e5bfe98698a7954f5ebe79804c3976
  Stored in directory: /root/.cache/pip/wheels/4c/a1/c6/b5697841db1112c6e5866d75a6b6bf1bef73b874782556ba66
Successfully built pretty-midi


In [ ]:
dataset_name = 'dataset_np.npy'
path = '/content/gdrive/MyDrive/Colab Notebooks/senior_research/'
path_progression = path+'GAN_progression2/'
path_model_backups = path+'GAN_models2/'
path_dataset = path+dataset_name

def load_real_samples():

    X = np.load(path_dataset)
    return X


dataset = load_real_samples()

In [ ]:
def define_discriminator(in_shape=(2,1024,128,1)):
    model = Sequential()
    model.add(Conv3D(64, (5,5,5), strides=(1, 2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv3D(64, (3,3,3), strides=(1, 2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv3D(64, (3,3,3), strides=(1, 2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    n_nodes = 2 * 64 * 8 * 32
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((2, 64, 8, 32)))
    model.add(Conv3DTranspose(64, (5,5,5), strides=(1,2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv3DTranspose(64, (2,2,2), strides=(1,2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv3DTranspose(128, (2,2,2), strides=(1,2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv3DTranspose(128, (2,2,2), strides=(1,2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv3D(1, (5,5,5), activation='sigmoid', padding='same'))
    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected midis
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples) 
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)[:,:,:,:,0]
    X = X*128
    X = np.floor(np.where(X < 30, 0, X))
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y

# create and save a plot of generated images (reversed grayscale)
def save_midis(examples, epoch, n=1):
    # save midis
    # midis = examples*128
    # midis = np.floor(np.where(midis < 1, 0, midis))
    midis=examples
    for i in range(n):
        name = 'generated_midi_e%03d_n%03d' % (epoch+1, i+1)
        #initialize
        multitrack = pypianoroll.Multitrack(name=name)
        #create left and right tracks
        right_track = pypianoroll.StandardTrack(name='right',pianoroll=midis[i,0])
        left_track = pypianoroll.StandardTrack(name='left',pianoroll=midis[i,1])
        #add tracks
        multitrack.append(right_track)
        multitrack.append(left_track)
        #save
        pypianoroll.write(path_progression+name+'.mid',multitrack)

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)



In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=4):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # create training set for the discriminator
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # update discriminator model weights
            if j % 30 == 0:
                d_loss, _ = d_model.train_on_batch(X[:,:,:,:,np.newaxis], y)
                # summarize loss on this batch
                print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
                # evaluate the model performance, sometimes
                if j == 0:
                    losses[i,0] = d_loss
                    losses[i,1] = g_loss
        if (i+1) % 1 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=10, n_samples_save=1):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real[:,:,:,:,np.newaxis], y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake[:,:,:,:,np.newaxis], y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save midis
    save_midis(x_fake, epoch, n_samples_save)
    # save the generator and discriminator models
    g_filename = 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(os.path.join(path_model_backups,g_filename))
    d_filename = 'discriminator_model_%03d.h5' % (epoch + 1)
    d_model.save(os.path.join(path_model_backups,d_filename))

n_epochs=500
losses = np.zeros((n_epochs,2))
# train model
train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=n_epochs, n_batch=10)

>1, 1/288, d=1.669, g=0.695
>1, 31/288, d=1.094, g=0.718
>1, 61/288, d=0.890, g=0.692
>1, 91/288, d=0.839, g=0.689
>1, 121/288, d=0.672, g=0.686
>1, 151/288, d=0.602, g=0.686
>1, 181/288, d=0.479, g=0.688
>1, 211/288, d=0.445, g=0.691
>1, 241/288, d=0.373, g=0.693
>1, 271/288, d=0.372, g=0.695
>Accuracy real: 100%, fake: 100%
>2, 1/288, d=0.538, g=0.696
>2, 31/288, d=0.386, g=0.698
>2, 61/288, d=0.363, g=0.700
>2, 91/288, d=0.348, g=0.704
>2, 121/288, d=0.349, g=0.708
>2, 151/288, d=0.352, g=0.713
>2, 181/288, d=0.343, g=0.719
>2, 211/288, d=0.345, g=0.725
>2, 241/288, d=0.347, g=0.731
>2, 271/288, d=0.392, g=0.737
>Accuracy real: 100%, fake: 30%
>3, 1/288, d=0.620, g=0.745
>3, 31/288, d=0.357, g=0.760
>3, 61/288, d=0.343, g=0.771
>3, 91/288, d=0.302, g=0.785
>3, 121/288, d=0.303, g=0.799
>3, 151/288, d=0.303, g=0.813
>3, 181/288, d=0.303, g=0.830


In [ ]:
def save_midis(examples, epoch, n=1):
    # save midis
    midis = examples*128
    midis = np.floor(np.where(midis < 1, 0, midis))
    for i in range(n):
        name = 'generated_midi_e%03d_n%03d' % (epoch+1, i+1)
        #initialize
        multitrack = pypianoroll.Multitrack(name=name)
        #create left and right tracks
        right_track = pypianoroll.StandardTrack(name='right',pianoroll=midis[i,0])
        left_track = pypianoroll.StandardTrack(name='left',pianoroll=midis[i,1])
        #add tracks
        multitrack.append(right_track)
        multitrack.append(left_track)
        #save
        pypianoroll.write(path_progression+name+'.mid',multitrack)
X_fake, y_fake = generate_fake_samples(g_model, 100, 1)
print(X_fake.shape)
save_midis(X_fake,0,1)
np.save('test',X_fake)

(1, 2, 1024, 128)
